In [ ]:
#install PennyLane Library
!pip install pennylane

In [ ]:
#import qml
import pennylane as qml


In [ ]:
#import the required packages
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

In [ ]:
#from tensorflow.keras.preprocessing import image
#from tensorflow.keras.preprocessing.image import load_img, img_to_array

 # # *Read COVID images dataset*

In [ ]:
#Function to read images dataset in grayscale
labells = ['NORMAL', 'COVID-19']
#labells = ['PNEUMONIA']
def get_PNEUMONIA_data(data_dir):
    data = [] 
    for label in labells: 
        path = os.path.join(data_dir, label)
        class_num = labells.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (28, 28)) 
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)


In [ ]:
#Train images
import cv2
train_PNEUMONIA = get_PNEUMONIA_data('/content/drive/My Drive/MS/Kaggle/PL/train')


In [ ]:
#Test images
testT_PNEUMONIA = get_PNEUMONIA_data('/content/drive/My Drive/MS/Kaggle/PL/test/')

In [ ]:
#Check number of images in Train and test
alldata2 = []
for i in testT_PNEUMONIA: #  for i in train_PNEUMONIA
    if(i[1] == 0):
        alldata2.append("Normal")
   # elif(i[1]==1):
        #alldata2.append("COVID-19")
    else:
        alldata2.append("COVID-19")
  
       
sns.set_style('darkgrid')
sns.countplot(alldata2)

In [ ]:
#check number and dimensions
testT_PNEUMONIA.shape

(385, 2)

In [ ]:

x_train_P = []
y_train_P = []

x_test_P = []
y_test_P = []

for feature, label in train_PNEUMONIA:
    x_train_P.append(feature)
    y_train_P.append(label)

for feature, label in testT_PNEUMONIA:
    x_test_P.append(feature)
    y_test_P.append(label)

In [ ]:
#x_train_PP = np.array(x_train_P) 


In [ ]:
x_train_PP = np.array(x_train_P) / 255
x_test_PP = np.array(x_test_P) / 255

In [ ]:
x_test_PP.shape

In [ ]:
y_train=np.array(y_train_P)
y_test=np.array(y_test_P)

In [ ]:
# Add new axis # Add extra dimension for convolution channels #train images
train_images_P55 = x_train_PP[..., tf.newaxis]


In [ ]:
# Add new axis # Add extra dimension for convolution channels #test images
train_images_P55 = x_train_PP[..., tf.newaxis]
test_images_P = x_test_PP[..., tf.newaxis]

In [ ]:
#Check dimension agter adding
test_images_P.shape

(385, 28, 28, 1)

In [ ]:
#train_imgs_L=np.asarray(train_imgs_L)
#test_imgs_L=np.asarray(test_imgs_L)

#Define Qunatum Device from PeenyLane with 1000 and 500 shots

In [ ]:

n_layers = 4  # Number of random layers
SAVE_PATH = "drive/My Drive/MS/Quantum_COVID_Data/" 
PREPROCESS = True          
np.random.seed(0)         
tf.random.set_seed(0)       

In [ ]:
train_it=test_imgs
result_test_imgs=train_it[:,:,:,0]

In [ ]:
#Quantum Device (default.qubit) with 1000 shots 
# For RX rotation gate
dev1 = qml.device("default.qubit",shots=1000, wires=4)
# Random circuit parameters
rand_params = np.random.uniform(high=2 * np.pi, size=(1, 4))

@qml.qnode(dev1)
def circuit_X(phi=None):
    # Encoding of 4 classical input values
    for j in range(4):
        qml.RX(np.pi * phi[j], wires=j)
    RandomLayers(rand_params, wires=list(range(4)))
    return [qml.expval(qml.PauliZ(j)) for j in range(4)]

In [ ]:
#Quantum Device (default.qubit) with 500 shots 
# For RY rotation gate
dev = qml.device("default.qubit",shots=500, wires=4)
# Random circuit parameters
rand_params = np.random.uniform(high=2 * np.pi, size=(1, 4))

@qml.qnode(dev)
def circuit(phi=None):
    # Encoding of 4 classical input values
    for j in range(4):
        qml.RY(np.pi * phi[j], wires=j)
    RandomLayers(rand_params, wires=list(range(4)))
    return [qml.expval(qml.PauliZ(j)) for j in range(4)]

# Quantum Circuit

In [ ]:
#Convolves the input image with the same quantum circuit 
def quanv(image):
    out = np.zeros((14, 14, 4))
    for j in range(0, 28, 2):
        for k in range(0, 28, 2):
            # Process a squared 2x2 region of the image with a quantum circuit
            q_results = circuit(
                phi=[image[j, k, 0], image[j, k + 1, 0], image[j + 1, k, 0], image[j + 1, k + 1, 0]]
            )
            # Assign expectation values to different channels of the output pixel (j/2, k/2)
            for c in range(4):
                out[j // 2, k // 2, c] = q_results[c]
    return out

In [ ]:
#Convolves the input image with the same quantum circuit 
def quanv2(image):
    out = np.zeros((14, 14, 4))
    for j in range(0, 28, 2):
        for k in range(0, 28, 2):
            # Process a squared 2x2 region of the image with a quantum circuit
            q_results = circuit_X(
                phi=[image[j, k, 0], image[j, k + 1, 0], image[j + 1, k, 0], image[j + 1, k + 1, 0]]
            )
            # Assign expectation values to different channels of the output pixel (j/2, k/2)
            for c in range(4):
                out[j // 2, k // 2, c] = q_results[c]
    return out

In [ ]:
#AA_test_imgs.shape

# Encoding Images  for quantum data using Rotation gates (RY and RX)

In [ ]:
# Encoding iamges for quantum data
import time 
start=time.time()  # for test time of encoding images to quantum data
if PREPROCESS == True:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(train_images_P55):
        #print("{}/{}        ".format(idx + 1, 536), end="\r")
        q_train_images.append(quanv(img))
    q_train_imagesZX2 = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(test_images_P):
       #print("{}/{}        ".format(idx + 1, 120), end="\r")
       q_test_images.append(quanv(img))
    q_test_imagesZX2 = np.asarray(q_test_images)
   
    # Save pre-processed images
    #np.save(SAVE_PATH + "q_train_images.npy", q_train_images)
    #np.save(SAVE_PATH + "q_test_images.npy", q_test_images)

print("time =",time.time()-start)
# Load pre-processed images
#q_train_images = np.load(SAVE_PATH + "q_train_images.npy")
#q_test_images = np.load(SAVE_PATH + "q_test_images.npy")

In [ ]:
import time 
start=time.time()
if PREPROCESS == True:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(train_images_P55):
        #print("{}/{}        ".format(idx + 1, 3336), end="\r")
        q_train_images.append(quanv2(img))
    q_train_imagesZZ_P2 = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(test_images_P):
        #print("{}/{}        ".format(idx + 1, 624), end="\r")
        q_test_images.append(quanv2(img))

    q_test_imagesZ2 = np.asarray(q_test_images)
print("Total time",time.time()-start)

In [ ]:
import time 
start=time.time()
if PREPROCESS == True:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(train_images_P55):
        #print("{}/{}        ".format(idx + 1, 3336), end="\r")
        q_train_images.append(quanv(img))
    q_train_imagesZZ_P = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(test_images_P):
        #print("{}/{}        ".format(idx + 1, 624), end="\r")
        q_test_images.append(quanv(img))

    q_test_imagesZ = np.asarray(q_test_images)
print("Total time",time.time()-start)

In [ ]:
#np.save(SAVE_PATH + "Q_P_Train.npy", q_train_imagesZZ_P)


In [ ]:

#np.save(SAVE_PATH + "Q_P_Test.npy", q_test_imagesZZ_P)

#Images after Encoding

In [ ]:
#Show images after quantum pre-processing
#PNEUMONIA Images
n_samples = 3
n_channels =4
plt.rcParams.update({'font.size': 15})
#plt.axis('off')
fig, axes = plt.subplots(1 + n_channels, n_samples, figsize=(12,12))
for k in range(n_samples):
    axes[0, 0].set_ylabel("Pn")
    if k != 0:
        axes[0, k].yaxis.set_visible(True)
    axes[0, k].imshow(AA_test_imgs[k, :, :, 0], cmap="gray")

    # Plot all output channels
    for c in range(n_channels):
        axes[c + 1, 0].set_ylabel("channel_{}".format(c+1))
        if k != 0:
            axes[c, k].yaxis.set_visible(True)
        axes[c + 1, k].imshow(q_test_imagesZX[k, :, :, c], cmap="RdGy")

plt.tight_layout()
plt.show()

In [ ]:
#Show images after quantum pre-processing
#COVID-19 Images
n_samples = 3
n_channels =4
plt.rcParams.update({'font.size': 15})
#plt.axis('off')
fig, axes = plt.subplots(1 + n_channels, n_samples, figsize=(12,12))
for k in range(n_samples):
    axes[0, 0].set_ylabel("COVID-19")
    if k != 0:
        axes[0, k].yaxis.set_visible(True)
    axes[0, k].imshow(AA_test_imgs[k, :, :, 0], cmap="gray")

    # Plot all output channels
    for c in range(n_channels):
        axes[c + 1, 0].set_ylabel("channel_{}".format(c+1))
        if k != 0:
            axes[c, k].yaxis.set_visible(True)
        axes[c + 1, k].imshow(q_test_imagesZX[k, :, :, c], cmap="RdGy")

plt.tight_layout()
plt.show()

In [ ]:
#Show images after quantum pre-processing
#Normal Images
n_samples = 3
n_channels =4
plt.rcParams.update({'font.size': 15})
#plt.axis('off')
fig, axes = plt.subplots(1 + n_channels, n_samples, figsize=(12,12))
for k in range(n_samples):
    axes[0, 0].set_ylabel(" Normal")
    if k != 0:
        axes[0, k].yaxis.set_visible(True)
    axes[0, k].imshow(AA_test_imgs[k, :, :, 0], cmap="gray")

    # Plot all output channels
    for c in range(n_channels):
        axes[c+1 , 0].set_ylabel("channel_{}".format(c+1))
        if k != 0:
            axes[c, k].yaxis.set_visible(True)
        axes[c + 1, k].imshow(q_test_imagesZX[k, :, :, c], cmap="RdGy")

plt.tight_layout()
plt.show()

In [ ]:
#from keras import backend as K  
#K.set_image_dim_ordering('tf')


#Create Quantum and Classical Models

In [ ]:
#create quantum model layers
def Quantum_Model():
    model = keras.models.Sequential([
       #keras.ZeroPadding2D(1,1),
       keras.layers.Conv2D(16, (2, 2), padding="same",activation='relu'),
       keras.layers.MaxPooling2D((2,2), strides=(2,2),padding="same"),   
       keras.layers.Dropout(0.2),                       
       keras.layers.Conv2D(16, (2, 2), padding="same", activation='relu'),
       keras.layers.Conv2D(32, (2, 2),padding="same", activation='relu'),
       keras.layers.MaxPooling2D((2,2), strides=(2,2)),
       keras.layers.Dropout(0.2),      
       keras.layers.Flatten(),
       #keras.layers.Dropout(0.4),
       keras.layers.Dense(300,activation = 'relu'),
       keras.layers.Dropout(0.5),
       #keras.layers.Dense(512,activation = 'relu'),
       #keras.layers.Dropout(0.4),
       keras.layers.Dense(100,activation = 'relu'),
       keras.layers.Dropout(0.4),
       #keras.layers.Dense(56,activation = 'relu'),
      #keras.layers.Dropout(0.02),
       keras.layers.Dense(2, activation="softmax")
       #for multi-class
       #keras.layers.Dense(3, activation="softmax")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(lr=0.0001),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
     )
    
    #model.compile(optimizer=RAdam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'],)
  
    return model

In [ ]:
 #create classical model layers
 def Classical_Model():
    model = keras.models.Sequential([
                                     
       keras.layers.Conv2D(4, (2, 2),padding="same", activation='relu'),
       #keras.layers.MaxPooling2D((2,2), strides=(2,2,),padding="same"),   
       keras.layers.Conv2D(16, (2, 2), padding="same",activation='relu'),
       keras.layers.MaxPooling2D((2,2), strides=(2,2,),padding="same"),   
       keras.layers.Dropout(0.2),           
       #keras.layers.MaxPooling2D((2,2), strides=(2,2),padding="same"), 
       keras.layers.Conv2D(16, (2, 2), padding="same", activation='relu'),
       keras.layers.Conv2D(32, (2, 2),padding="same", activation='relu'),
       keras.layers.MaxPooling2D((2,2), strides=(2,2)),
       keras.layers.Dropout(0.2),           
       keras.layers.Flatten(),
       keras.layers.Dense(300,activation = 'relu'),
       keras.layers.Dropout(0.5),
       keras.layers.Dense(100,activation = 'relu'),
       keras.layers.Dropout(0.4),
       #keras.layers.Dense(56,activation = 'relu'),
       #keras.layers.Dropout(0.5),
       keras.layers.Dense(2, activation="softmax")
       #for multi-class
       #keras.layers.Dense(3, activation="softmax")
  
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(lr=0.0001),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    #model.compile(optimizer=RAdam(), loss='binary_crossentropy', metrics=['accuracy'])

    return model 

In [ ]:
 def MClassical_Model():
  
    model = keras.models.Sequential([
                                     
       keras.layers.Flatten(),
       keras.layers.Dense(300,activation = 'relu'),
       #keras.layers.Dropout(0.4),
       keras.layers.Dense(100,activation = 'relu'),
       #keras.layers.Dropout(0.5),
       keras.layers.Dense(2, activation="softmax")
       #for multi-class
       #keras.layers.Dense(3, activation="softmax")  
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(lr=0.0001),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )


# ML Algorithms

In [ ]:
from keras.models import Model

## Feat Classical Model

In [ ]:
#Feat classical model to use ML algorithms instead of softmax layer
model_feat = Model(inputs=VVCmodel.input,outputs=VVCmodel.get_layer('dense_34').output)
feat_train = model_feat.predict(train_images_P55)

In [ ]:
#SVM
#Train
from sklearn.svm import SVC
svm = SVC(kernel='rbf',probability=True)
#svm.fit(feat_train,np.argmax(y_train_PP,axis=1))
svm.fit(feat_train,y_train)

In [ ]:
#test
feat_test = model_feat.predict(test_images_P)


In [ ]:
svm.score(feat_test,y_test)


In [ ]:
from sklearn.svm import SVC
svmPoly = SVC(kernel='poly',probability=True)
svmPoly.fit(feat_train,y_train)

In [ ]:
svmPoly.score(feat_test,y_test)


In [ ]:
#from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(feat_train,y_train)


In [ ]:
neigh.score(feat_test,y_test)


In [ ]:
from sklearn.ensemble import BaggingClassifier
BC = BaggingClassifier(base_estimator=SVC(),
                         n_estimators=3, random_state=0).fit(feat_train, y_train)

In [ ]:
BC.score(feat_test,y_test)


0.89171974522293

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ABC=AdaBoostClassifier( n_estimators=50)
ABC.fit(feat_train,y_train)


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)

In [ ]:
ABC.score(feat_test,y_test)


In [ ]:
from sklearn.tree import DecisionTreeClassifier 
DT = DecisionTreeClassifier(max_depth=5) 
DT.fit(feat_train,y_train)

In [ ]:
DT.score(feat_test,y_test)


In [ ]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB() 
gnb.fit(feat_train,y_train)
gnb.score(feat_test,y_test)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators =5)
RF.fit(feat_train,y_train)


In [ ]:
RF.score(feat_test,y_test)


In [ ]:
import xgboost as xgb
xgbc = xgb.XGBClassifier(max_depth=1).fit(feat_train, y_train)
#predictions = xgbc.predict(feat_test)


In [ ]:
xgbc.score(feat_test,y_test)


#Train and Test

In [ ]:

start=time.time()
VCmodel=Classical_Model()
VC_history = VCmodel.fit(
    train_images_P55,
    y_train,
    validation_data=(test_images_P, y_test),
    batch_size=32,
    epochs=100,
    verbose=2)
print("Time=",time.time()-start)

In [ ]:
q_model = Quantum_Model()
start=time.time()
q_history = q_model.fit(
    q_train_imagesZZ_P,
    y_train,
    validation_data=(q_test_imagesZ, y_test),
    batch_size=32,
    epochs=100,  
   verbose=2,
)
print("Time=",time.time()-start)

In [ ]:
q_modelx2 = Quantum_Model()
start=time.time()
q_historyx2 = q_modelx2.fit(
    q_train_imagesZX2,
    y_train,
    validation_data=(q_test_imagesZX2, y_test),
    batch_size=32,
    epochs=100,  
   verbose=2,
)
print("Time=",time.time()-start)

In [ ]:
# RX gates
q_modelx = Quantum_Model()
start=time.time()
q_historyx = q_modelx.fit(
    q_train_imagesZX,
    y_train,
    validation_data=(q_test_imagesZX, y_test),
    batch_size=32,
    epochs=20,  
   verbose=2,
)
print("Time=",time.time()-start)

In [ ]:
#q_train_imagesZZ_P.shape

In [ ]:
# RY gate 
q_model = Quantum_Model()
start=time.time()
q_history = q_model.fit(
    q_train_imagesZZ_P,
    y_train,
    validation_data=(q_test_imagesZ, y_test),
    batch_size=32,
    epochs=20,  
   verbose=2,
)
print("Time=",time.time()-start)

In [ ]:
q_model2 = Quantum_Model()
start=time.time()
q_history2 = q_model2.fit(
    q_train_imagesZZ_P2,
    y_train,
    validation_data=(q_test_imagesZ2, y_test),
    batch_size=32,
    epochs=20,  
   verbose=2,
)
print("Time=",time.time()-start)

In [ ]:
mlp=MClassical_Model()
mlp_history = mlp.fit(
    train_images_P55,
    y_train,
    validation_data=(test_images_P, y_test),
    batch_size=32,
    epochs=100,
    verbose=2)

In [ ]:
q_model.summary()

In [ ]:
q_modelx2.summary()

In [ ]:
VCmodel.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(q_modelx2, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
plot_model(VCmodel, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


Accuary and Loss RY and RX gates

In [ ]:
epochs=20
#acc=q_history.history['accuracy']  
epochs_=range(0,epochs)    
plt.plot(epochs_,q_history.history["val_accuracy"],'b' , label=" Test Accuarcy + HQCNN+RY+500 shots ")
plt.plot(epochs_,q_history2.history["val_accuracy"],'r',  label=" Test Accuarcy + HQCNN+RY+1000 shots ")
plt.plot(epochs_,q_historyx.history["val_accuracy"], 'g', label=" Test Accuarcy + HQCNN+RX+500 shots ")
plt.plot(epochs_,q_historyx2.history["val_accuracy"], 'c', label=" Test Accuarcy + HQCNN+RX+1000 shots")
plt.ylabel("Accuracy")
#plt.ylim([60,100])
plt.xlim([0,20])

plt.xlabel("Epoch")
plt.legend(loc="best")


In [ ]:
import pandas as pd 
import numpy as np
train_imgs_cls=list(train_imgs)

In [ ]:
epochs=20
acc=c_history.history['accuracy']  ##getting  accuracy of each epochs
epochs_=range(0,epochs)    
plt.plot(epochs_,acc,label='training accuracy')
plt.xlabel('No of Epochs')
plt.ylabel('Accuracy')

acc_val=c_history.history['val_accuracy']  ##getting validation accuracy of each epochs
plt.plot(epochs_,acc_val,label="validation accuracy")
plt.title("No of Epochs vs Accuracy")
plt.legend()

In [ ]:
predc = q_model.predict(q_test_imagesZ)

In [ ]:
y = np.argmax(y_test, axis=1)

In [ ]:
preds = np.argmax(predc, axis=1)
cm = confusion_matrix(y_test, preds)
print(cm)

In [ ]:
preds

In [ ]:
import pandas as pd
x_test=pd.DataFrame(x_test)

In [ ]:
predCls = q_modelx.predict(q_test_imagesZX)
#converting Probabilities to labels
predCls = np.argmax(predCls, axis=1)
#label
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score#Accuracy Score
print ('Accuracy Score : ', accuracy_score(y_test,predCls,), '\n')

#precision, recall report
print ('Classification Report :\n\n' ,classification_report(y_test, predCls))

In [ ]:
predCls2 = q_modelx2.predict(q_test_imagesZX2)
#pred
#converting Probabilities to labels
predCls2 = np.argmax(predCls2, axis=1)
#label
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score#Accuracy Score
print ('Accuracy Score : ', accuracy_score(y_test,predCls2,), '\n')

#precision, recall report
print ('Classification Report :\n\n' ,classification_report(y_test, predCls2))

In [ ]:
#prediction on test set
pred2 = q_model2.predict(q_test_imagesZ2)
#converting Probabilities to labels
label2 = np.argmax(pred2, axis=1)
#label
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score#Accuracy Score
print ('Accuracy Score : ', accuracy_score(y_test,label2), '\n')

#precision, recall report
print ('Classification Report :\n\n' ,classification_report(label2, y_test))

In [ ]:
#prediction on test set
pred = q_model.predict(q_test_imagesZ)
#converting Probabilities to labels
label = np.argmax(pred, axis=1)
#label
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score#Accuracy Score
print ('Accuracy Score : ', accuracy_score(label,y_test), '\n')

#precision, recall report
print ('Classification Report :\n\n' ,classification_report(label, y_test))

# Confusion *Matrcies*

In [ ]:
CMQ2=metrics.confusion_matrix(y_test,label2)
CMQ2

In [ ]:
import sklearn.metrics as metrics

CMQ=metrics.confusion_matrix(y_test,label)
CMQ

## RX and RX  with 500 and 1000 shots

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
#fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(CMQ2, annot=True,fmt="g")
ax= plt.subplot()
plt.title("CM_HQCNN+RY+1000 shots")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
plt.show()

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
#fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(CMQ, annot=True,fmt="g")
ax= plt.subplot()
plt.title("CM_HQCNN+RY+500 shots")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);

plt.show()



In [ ]:
CMCnn=metrics.confusion_matrix(y_test,predCls)
CMCnn

In [ ]:
CMCnn2=metrics.confusion_matrix(y_test,predCls2)
CMCnn2

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
sns.heatmap(CMCnn2, annot=True,fmt="g")
ax= plt.subplot()
plt.title("CM_HQCNN+RX+1000 shots")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#s = [['TN','FP'], ['FN', 'TP']]
#for i in range(2):
    #for j in range(2):
        #plt.text(j,i, str(s[i][j])+" = "+str(confusion_matrix[i][j]))
plt.show()

## HQCNN and CNN model

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
sns.heatmap(CMQ, annot=True,fmt="g")
ax= plt.subplot()
plt.title("CM_HQCNN+RX+500 shots")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#s = [['TN','FP'], ['FN', 'TP']]
plt.show()



In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
sns.heatmap(CMCnn, annot=True,fmt="g")
ax= plt.subplot()
plt.title("CM_HQCNN")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#s = [['TN','FP'], ['FN', 'TP']]
plt.show()

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
sns.heatmap(CMCnn, annot=True,fmt="g")
ax= plt.subplot()
plt.title("CM_CNN")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#s = [['TN','FP'], ['FN', 'TP']]
plt.show()

## Other ML Algorithms for comparsion

In [ ]:
pred_mlp = mlp.predict(test_images_P)
#pred

#converting Probabilities to labels
pred_mlp = np.argmax(pred_mlp, axis=1)
CMMLP=metrics.confusion_matrix(y_test,pred_mlp)
CMMLP

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
sns.heatmap(CMMLP, annot=True,fmt="g")
ax= plt.subplot()
plt.title("CM_MLP")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
plt.show()

In [ ]:
predsvm = svm.predict(feat_test)

CMSVM=metrics.confusion_matrix(y_test,predsvm)
CMSVM

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
sns.heatmap(CMSVM, annot=True,fmt="g")
ax= plt.subplot()
plt.title("CM_CNN+SVM (RBF)")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
plt.show()


In [ ]:
predsvmP = svmPoly.predict(feat_test)

CMSVM_P=metrics.confusion_matrix(y_test,predsvmP)
CMSVM_P

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
sns.heatmap(CMSVM_P, annot=True,fmt="g")
ax= plt.subplot()
plt.title("CM_CNN+SVM (Poly)")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
plt.show()


In [ ]:
predKNN = neigh.predict(feat_test)
CMKNN=metrics.confusion_matrix(y_test,predKNN)
CMKNN

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
sns.heatmap(CMKNN, annot=True,fmt="g")
ax= plt.subplot()
plt.title("CM_CNN+KNN")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
plt.show()

In [ ]:
predABC = ABC.predict(feat_test)
CMABC=metrics.confusion_matrix(y_test,predABC)
CMABC

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
sns.heatmap(CMABC, annot=True,fmt='g')
ax= plt.subplot()
plt.title("CM_CNN+AdaBoost")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
plt.show()

In [ ]:
predRF = RF.predict(feat_test)
CMRF=metrics.confusion_matrix(y_test,predRF)
CMRF

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
sns.heatmap(CMRF, annot=True,fmt='g')
ax= plt.subplot()
plt.title("CM_CNN+RF")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
plt.show()

In [ ]:
predDT = DT.predict(feat_test)
CMDT=metrics.confusion_matrix(y_test,predDT)
CMDT

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
sns.heatmap(CMDT, annot=True,fmt='g')
ax= plt.subplot()
plt.title("CM_CNN+DT")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
plt.show()

In [ ]:
prednb = gnb.predict(feat_test)
CMNB=metrics.confusion_matrix(y_test,prednb)
CMNB

In [ ]:
predmnb = BC.predict(feat_test)
CM_MNB=metrics.confusion_matrix(y_test,predmnb)
CM_MNB

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
sns.heatmap(CMNB, annot=True,fmt='g')
ax= plt.subplot()
plt.title("CM_CNN+GNB")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
plt.show()

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
sns.heatmap(CM_MNB, annot=True,fmt='g')
ax= plt.subplot()
plt.title("CM_CNN+BC")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
plt.show()

In [ ]:
predXGB = xgbc.predict(feat_test)
CMXGB=metrics.confusion_matrix(y_test,predXGB)
CMXGB

In [ ]:
import seaborn as sns
sns.set(font_scale=1.1)
#cmap=plt.cm.RdGy_r
sns.heatmap(CMXGB, annot=True,fmt='g')
ax= plt.subplot()
plt.title("CM_CNN+XGB")
#plt.tight_layout()
plt.ylabel(' True Label')
plt.xlabel(' Predicted Label ')
ax.xaxis.set_ticklabels(['Normal', 'COVID-19']); ax.yaxis.set_ticklabels(['Normal', 'COVID-19']);
#ax.xaxis.set_ticklabels(['COVID-19', 'Pneumonia']); ax.yaxis.set_ticklabels(['COVID-19', 'Pneumonia']);
#ax.xaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);
#ax.yaxis.set_ticklabels(['Normal','COVID-19', 'Pneumonia']);

plt.show()

In [ ]:
results=[1,23,33]
names=["n","m","k"]
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
for i,j in results:
  plt.boxplot(results)
  ax.set_xticklabels(names)
  plt.show()

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, label))
print("RMSE: %f" % (rmse))

# Evaluation Scores


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import fbeta_score

## HQCNN and CNN Models

In [ ]:
###########################  HQCNN ##############
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test,label2)
print('Accuracy: %f' % accuracy)

accuracy = balanced_accuracy_score(y_test,label2)
print('Balanced_Accuracy: %f' % accuracy)
Pre=fbeta_score(y_test, label2, average='weighted', beta=2)
rec=fbeta_score(y_test, label2, average='weighted', beta=0.5)
print('Pre: %f' % Pre)
print('rec: %f' % rec)

# precision tp / (tp + fp)
precision = precision_score(y_test,label2)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test,label2)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,label2)
print('F1 score: %f' % f1)

In [ ]:
accuracy = accuracy_score(y_test,predCls2)
print('Accuracy: %f' % accuracy)
##
Pre=fbeta_score(y_test, predCls2, average='weighted', beta=2)
rec=fbeta_score(y_test, predCls2, average='weighted', beta=0.5)
print('Pre: %f' % Pre)
print('rec: %f' % rec)
accuracy = balanced_accuracy_score(y_test,predCls2)
print('Balanced_Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test,predCls2)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test,predCls2)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,predCls2)
print('F1 score: %f' % f1)

In [ ]:
###########################  HQCNN ##############
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test,label)
print('Accuracy: %f' % accuracy)

accuracy = balanced_accuracy_score(y_test,label)
print('Balanced_Accuracy: %f' % accuracy)
Pre=fbeta_score(y_test, label, average='weighted', beta=2)
rec=fbeta_score(y_test, label, average='weighted', beta=0.5)
print('Pre: %f' % Pre)
print('rec: %f' % rec)

# precision tp / (tp + fp)
precision = precision_score(y_test,label)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test,label)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,label)
print('F1 score: %f' % f1)

# kappa
#kappa = cohen_kappa_score(y_test,label)
#print('Cohens kappa: %f' % kappa)
# ROC AUC
#auc = roc_auc_score(y_test,label)
#print('ROC AUC: %f' % auc)
# confusion matrix
#matrix = confusion_matrix(y_test,label)
#print(matrix)

In [ ]:
###########################  CNN ##############
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test,predCls)
print('Accuracy: %f' % accuracy)
##
Pre=fbeta_score(y_test, predCls, average='weighted', beta=2)
rec=fbeta_score(y_test, predCls, average='weighted', beta=0.5)
print('Pre: %f' % Pre)
print('rec: %f' % rec)
accuracy = balanced_accuracy_score(y_test,predCls)
print('Balanced_Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test,predCls)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test,predCls)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,label)
print('F1 score: %f' % f1)

# kappa
#kappa = cohen_kappa_score(y_test,predCls)
#print('Cohens kappa: %f' % kappa)
# ROC AUC
#auc = roc_auc_score(y_test,predCls)
#print('ROC AUC: %f' % auc)
# confusion matrix
#matrix = confusion_matrix(y_test,label)
#print(matrix)

## Other ML Algorithms for comparsion

In [ ]:
# average 'weighted' is addes with Multi-class dataset

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test,pred_mlp)
print('Accuracy: %f' % accuracy)
accuracy = balanced_accuracy_score(y_test,pred_mlp)
print('Balanced_Accuracy: %f' % accuracy)

Pre=fbeta_score(y_test, pred_mlp, average='weighted', beta=2)
rec=fbeta_score(y_test, pred_mlp, average='weighted', beta=0.5)
print('Pre: %f' % Pre)
print('rec: %f' % rec)
# precision tp / (tp + fp)
precision = precision_score(y_test,pred_mlp,average="weighted")
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test,pred_mlp,average="weighted")
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,pred_mlp,average="weighted")
print('F1 score: %f' % f1)

In [ ]:
###########################  SVM_RBF ##############
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test,predsvm)
print('Accuracy: %f' % accuracy)

accuracy = balanced_accuracy_score(y_test,predsvm)
print('Balanced_Accuracy: %f' % accuracy)

Pre=fbeta_score(y_test, predsvm, average='weighted', beta=2)
rec=fbeta_score(y_test, predsvm, average='weighted', beta=0.5)
print('Pre: %f' % Pre)
print('rec: %f' % rec)

# precision tp / (tp + fp)
precision = precision_score(y_test,predsvm,average="weighted")
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test,predsvm,average="weighted")
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,predsvm,average="weighted")
print('F1 score: %f' % f1)



In [ ]:
###########################  SVM_POLY ##############
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test,predsvmP)
print('Accuracy: %f' % accuracy)
accuracy = balanced_accuracy_score(y_test,predsvmP)
print('Balanced_Accuracy: %f' % accuracy)

Pre=fbeta_score(y_test, predsvmP, average='weighted', beta=2)
rec=fbeta_score(y_test, predsvmP, average='weighted', beta=0.5)
print('Pre: %f' % Pre)
print('rec: %f' % rec)
# precision tp / (tp + fp)
precision = precision_score(y_test,predsvmP,average="weighted")
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test,predsvmP,average="weighted")
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,predsvmP,average="weighted")
print('F1 score: %f' % f1)



In [ ]:
###########################  KNN ##############
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test,predKNN)
print('Accuracy: %f' % accuracy)

accuracy = balanced_accuracy_score(y_test,predKNN)
print('Balanced_Accuracy: %f' % accuracy)

Pre=fbeta_score(y_test, predKNN, average='weighted', beta=2)
rec=fbeta_score(y_test, predKNN, average='weighted', beta=0.5)
print('Pre: %f' % Pre)
print('rec: %f' % rec)

# precision tp / (tp + fp)
precision = precision_score(y_test,predKNN,average="weighted")
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test,predKNN,average="weighted")
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,predKNN,average="weighted")
print('F1 score: %f' % f1)


In [ ]:
###########################  ABC ##############
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, predABC)
print('Accuracy: %f' % accuracy)

accuracy = balanced_accuracy_score(y_test,predABC)
print('Balanced_Accuracy: %f' % accuracy)

Pre=fbeta_score(y_test, predABC, average='weighted', beta=2)
rec=fbeta_score(y_test, predABC, average='weighted', beta=0.5)
print('Pre: %f' % Pre)
print('rec: %f' % rec)
# precision tp / (tp + fp)
precision = precision_score(y_test,predABC,average="weighted")
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test,predABC,average="weighted")
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,predABC,average="weighted")
print('F1 score: %f' % f1)



In [ ]:
###########################  RF ##############
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test,predRF)
print('Accuracy: %f' % accuracy)

accuracy = balanced_accuracy_score(y_test,predRF)
print('Balanced_Accuracy: %f' % accuracy)

Pre=fbeta_score(y_test, predRF, average='weighted', beta=2)
rec=fbeta_score(y_test, predRF, average='weighted', beta=0.5)
print('Pre: %f' % Pre)
print('rec: %f' % rec)
# precision tp / (tp + fp)
precision = precision_score(y_test,predRF,average="weighted")
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test,predRF,average="weighted")
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,predRF,average="weighted")
print('F1 score: %f' % f1)


In [ ]:
###########################  DT ##############
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test,predDT)
print('Accuracy: %f' % accuracy)

accuracy = balanced_accuracy_score(y_test,predDT)
print('Balanced_Accuracy: %f' % accuracy)

Pre=fbeta_score(y_test, predDT, average='weighted', beta=2)
rec=fbeta_score(y_test, predDT, average='weighted', beta=0.5)
print('Pre: %f' % Pre)
print('rec: %f' % rec)
# precision tp / (tp + fp)
precision = precision_score(y_test,predDT,average="weighted")
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test,predDT,average="weighted")
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,predDT,average="weighted")
print('F1 score: %f' % f1)



In [ ]:
###########################  NB ##############
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test,prednb)
print('Accuracy: %f' % accuracy)

accuracy = balanced_accuracy_score(y_test,prednb)
print('Balanced_Accuracy: %f' % accuracy)

Pre=fbeta_score(y_test, prednb, average='weighted', beta=2)
rec=fbeta_score(y_test, prednb, average='weighted', beta=0.5)
print('Pre: %f' % Pre)
print('rec: %f' % rec)
# precision tp / (tp + fp)
precision = precision_score(y_test,prednb,average="weighted")
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test,prednb,average="weighted")
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,prednb,average="weighted")
print('F1 score: %f' % f1)



In [ ]:
accuracy = accuracy_score(y_test,predmnb)
print('Accuracy: %f' % accuracy)

accuracy = balanced_accuracy_score(y_test,predmnb)
print('Balanced_Accuracy: %f' % accuracy)

Pre=fbeta_score(y_test, predmnb, average='weighted', beta=2)
rec=fbeta_score(y_test, predmnb, average='weighted', beta=0.5)
print('Pre: %f' % Pre)
print('rec: %f' % rec)
# precision tp / (tp + fp)
precision = precision_score(y_test,predmnb,average="weighted")
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test,predmnb,average="weighted")
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,predmnb,average="weighted")
print('F1 score: %f' % f1)

In [ ]:
###########################  XGB ##############
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test,predXGB)
print('Accuracy: %f' % accuracy)

accuracy = balanced_accuracy_score(y_test,predXGB)
print('Balanced_Accuracy: %f' % accuracy)

Pre=fbeta_score(y_test, predXGB, average='weighted', beta=2)
rec=fbeta_score(y_test, predXGB, average='weighted', beta=0.5)
print('Pre: %f' % Pre)
print('rec: %f' % rec)
# precision tp / (tp + fp)
precision = precision_score(y_test,predXGB,average="weighted")
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test,predXGB,average="weighted")
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,predXGB,average="weighted")
print('F1 score: %f' % f1)


# ROC Curve


## Binary Classification

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

y_pred_Q = q_model.predict(q_test_imagesZ)[:,1]
fpr_Q, tpr_Q, thresholds_cnn = roc_curve(y_test, y_pred_Q)
auc_Q= auc(fpr_Q, tpr_Q)

In [ ]:
y_pred_Q2 = q_model2.predict(q_test_imagesZ2)[:,1]
fpr_Q2, tpr_Q2, thresholds_cnn = roc_curve(y_test, y_pred_Q2)
auc_Q2= auc(fpr_Q2, tpr_Q2)

In [ ]:
y_pred_C = q_modelx.predict_proba(q_test_imagesZX)[:,1]
#y_pred_MLP = mlp.predict_proba(test_images_P)
#y_pred_svm = svm.predict_proba(feat_test)
#y_pred_svm = svm.predict_proba(feat_test)


In [ ]:
y_pred_C2 = q_modelx2.predict(q_test_imagesZX2)[:,1]
fpr_cnn2, tpr_cnn2, thresholds_cnn2 = roc_curve(y_test, y_pred_C2)
auc_C2= auc(fpr_cnn2, tpr_cnn2)

In [ ]:
#y_pred_C = VCmodel.predict_proba(test_images_P)[:,1]
fpr_cnn, tpr_cnn, thresholds_cnn = roc_curve(y_test, y_pred_C)
auc_C= auc(fpr_cnn, tpr_cnn)

In [ ]:
y_pred_MLP = mlp.predict_proba(test_images_P)[:,1]
fpr_mlp, tpr_mlp, thresholds_mlp = roc_curve(y_test, y_pred_MLP,pos_label=3)
auc_mlp= auc(fpr_mlp, tpr_mlp)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_ranking.py:808: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


In [ ]:
y_pred_svmP = svmPoly.predict_proba(feat_test)
y_pred_knn = neigh.predict_proba(feat_test)
y_pred_abc = ABC.predict_proba(feat_test)
y_pred_rf = RF.predict_proba(feat_test)
###
y_pred_dt = DT.predict_proba(feat_test)
y_pred_nb = gnb.predict_proba(feat_test)
y_pred_mnb = BC.predict_proba(feat_test)
y_pred_xgb = xgbc.predict_proba(feat_test)


In [ ]:
y_pred_svm = svm.predict_proba(feat_test)[:,1]
fpr_svm, tpr_svm, thresholds_svm = roc_curve(y_test, y_pred_svm,pos_label=3)
auc_svm= auc(fpr_svm, tpr_svm)
###
y_pred_svmP = svmPoly.predict_proba(feat_test)[:,1]
fpr_svmP, tpr_svmP, thresholds_svmP = roc_curve(y_test, y_pred_svmP)
auc_svmP= auc(fpr_svmP, tpr_svmP)
##
y_pred_knn = neigh.predict_proba(feat_test)[:,1]

fpr_knn, tpr_knn, thresholds_KNN = roc_curve(y_test, y_pred_knn)
auc_KNN= auc(fpr_knn, tpr_knn)
###
y_pred_abc = ABC.predict_proba(feat_test)[:,1]

fpr_ABC, tpr_ABC, thresholds_ABC = roc_curve(y_test, y_pred_abc)
auc_ABC= auc(fpr_ABC, tpr_ABC)
###
y_pred_rf = RF.predict_proba(feat_test)[:,1]

fpr_RF, tpr_RF, thresholds_RF = roc_curve(y_test, y_pred_rf)
auc_RF= auc(fpr_RF, tpr_RF)
###
y_pred_dt = DT.predict_proba(feat_test)[:,1]

fpr_DT, tpr_DT, thresholds_DT = roc_curve(y_test, y_pred_dt)
auc_DT= auc(fpr_DT, tpr_DT)
###
y_pred_nb = gnb.predict_proba(feat_test)[:,1]

fpr_GNB, tpr_GNB, thresholds_GNB = roc_curve(y_test, y_pred_nb)
auc_GNB= auc(fpr_GNB, tpr_GNB)
###
y_pred_mnb = BC.predict_proba(feat_test)[:,1]

fpr_mnb, tpr_mnb, thresholds_mnb = roc_curve(y_test, y_pred_mnb)
auc_mnb= auc(fpr_mnb, tpr_mnb)
###
y_pred_xgb = xgbc.predict_proba(feat_test)[:,1]
fpr_XGB, tpr_XGB, thresholds_XGB = roc_curve(y_test, y_pred_xgb)
auc_XGB= auc(fpr_XGB, tpr_XGB)


## Multi-class Classification

In [ ]:
from sklearn.preprocessing import label_binarize

Y = label_binarize(y_test,classes=[0, 1, 2])


In [ ]:
from itertools import cycle


In [ ]:
classes=[0, 1, 2]
fpr_Q = dict()
tpr_Q = dict()
auc_Q = dict()
fpr_cnn=dict()
tpr_cnn=dict()
auc_c=dict()
fpr_mlp=dict()
tpr_mlp=dict()
auc_mlp=dict()
fpr_svm=dict()
tpr_svm=dict()
auc_svm=dict()
plt.style.use('seaborn-white')

plt.figure(figsize=(8,9))
#plt.rcParams['axes.facecolor'] = 'k'
#plt.facecolor="w"
#plt.axes.color="k"
plt.plot([0, 1], [0, 1], 'k--')
for i in range(3):
    fpr_Q[i], tpr_Q[i], thresholds_Q = roc_curve(Y[:,i], y_pred_Q[:,i])
    auc_Q[i] = auc(fpr_Q[i], tpr_Q[i])
    fpr_cnn[i], tpr_cnn[i], thresholds_cnn = roc_curve(Y[:,i], y_pred_C[:,i])
    auc_c[i]= auc(fpr_cnn[i], tpr_cnn[i])
    fpr_mlp[i], tpr_mlp[i], thresholds_mlp = roc_curve(Y[:,i], y_pred_MLP[:,i])
    auc_mlp[i]= auc(fpr_mlp[i], tpr_mlp[i])
    fpr_svm[i], tpr_svm[i], thresholds_svm = roc_curve(Y[:,i], y_pred_svm[:,i])
    auc_svm[i]= auc(fpr_svm[i], tpr_svm[i])
    #colors = cycle(['r', 'b', 'tab:olive','c','k','g','y','tab:pink','tab:orange','tab:purple','m','tab:brown'])
    colors = cycle(['b', 'r', 'c'])
    colors1 = cycle(['k','g','y'])
    colors2 = cycle(['m','tab:pink' ,'tab:brown'])
    colors3 = cycle(['tab:olive','tab:orange','tab:purple'])



for i, color in zip(range(3), colors):
    plt.plot( fpr_Q[i], tpr_Q[i], color=color,
             label='HQCNN class %i (area = %0.3f)'% (i,auc_Q[i]))
for i, color in zip(range(3), colors1):
    plt.plot( fpr_cnn[i], tpr_cnn[i], color=color,
             label='CNN class %i (area = %0.3f)'% (i,auc_c[i] ))
for i, color in zip(range(3), colors2):
    plt.plot( fpr_mlp[i], tpr_mlp[i], color=color,
             label='MLP class %i (area = %0.3f)'% (i,auc_mlp[i] ))
for i, color in zip(range(3), colors3):
    plt.plot( fpr_svm[i], tpr_svm[i], color=color,
                label='CNN+SVM(rbf) class %i (area = %0.3f)'% (i,auc_svm[i] )) 
    
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('AUC-ROC Curve')
plt.legend(loc='best')

plt.show()


In [ ]:
classes=[0, 1, 2]
fpr_svmP= dict()
tpr_svmP = dict()
auc_svmP = dict()
fpr_knn=dict()
tpr_knn=dict()
auc_knn=dict()
fpr_abc=dict()
tpr_abc=dict()
auc_abc=dict()
fpr_rf=dict()
tpr_rf=dict()
auc_rf=dict()
plt.style.use('seaborn-white')

plt.figure(figsize=(8,9))
#plt.rcParams['axes.facecolor'] = 'k'
#plt.facecolor="w"
#plt.axes.color="k"
plt.plot([0, 1], [0, 1], 'k--')
for i in range(3):
    fpr_svmP[i], tpr_svmP[i], thresholds_Q = roc_curve(Y[:,i], y_pred_svmP[:,i])
    auc_svmP[i] = auc(fpr_svmP[i], tpr_svmP[i])
    fpr_knn[i], tpr_knn[i], thresholds_cnn = roc_curve(Y[:,i], y_pred_knn[:,i])
    auc_knn[i]= auc(fpr_knn[i], tpr_knn[i])
    fpr_abc[i], tpr_abc[i], thresholds_mlp = roc_curve(Y[:,i], y_pred_abc[:,i])
    auc_abc[i]= auc(fpr_abc[i], tpr_abc[i])
    fpr_rf[i], tpr_rf[i], thresholds_svm = roc_curve(Y[:,i], y_pred_rf[:,i])
    auc_rf[i]= auc(fpr_rf[i], tpr_rf[i])
    #colors = cycle(['r', 'b', 'tab:olive','c','k','g','y','tab:pink','tab:orange','tab:purple','m','tab:brown'])
    colors = cycle(['b', 'r', 'c'])
    colors1 = cycle(['k','g','y'])
    colors2 = cycle(['m','tab:pink' ,'tab:brown'])
    colors3 = cycle(['tab:olive','tab:orange','tab:purple'])



for i, color in zip(range(3), colors):
    plt.plot( fpr_svmP[i], tpr_svmP[i], color=color,
             label='CNN+SVM(Poly) class %i (area = %0.3f)'% (i,auc_svmP[i]))
for i, color in zip(range(3), colors1):
    plt.plot( fpr_knn[i], tpr_knn[i], color=color,
             label='CNN+KNN class %i (area = %0.3f)'% (i,auc_knn[i] ))
for i, color in zip(range(3), colors2):
    plt.plot( fpr_abc[i], tpr_abc[i], color=color,
             label='CNN+AdaBoost class %i (area = %0.3f)'% (i,auc_abc[i] ))
for i, color in zip(range(3), colors3):
    plt.plot( fpr_rf[i], tpr_rf[i], color=color,
                label='CNN+RF class %i (area = %0.3f)'% (i,auc_rf[i] )) 
    
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('AUC-ROC Curve')
plt.legend(loc='best')

plt.show()


In [ ]:
classes=[0, 1, 2]
fpr_dt = dict()
tpr_dt = dict()
auc_dt = dict()
fpr_nb=dict()
tpr_nb=dict()
auc_nb=dict()
fpr_bc=dict()
tpr_bc=dict()
auc_bc=dict()
fpr_xgb=dict()
tpr_xgb=dict()
auc_xgb=dict()
plt.style.use('seaborn-white')

plt.figure(figsize=(8,9))
#plt.rcParams['axes.facecolor'] = 'k'
#plt.facecolor="w"
#plt.axes.color="k"
plt.plot([0, 1], [0, 1], 'k--')
for i in range(3):
    fpr_bc[i], tpr_bc[i], thresholds_Q = roc_curve(Y[:,i], y_pred_mnb[:,i])
    auc_bc[i] = auc(fpr_bc[i], tpr_bc[i])
    fpr_dt[i], tpr_dt[i], thresholds_cnn = roc_curve(Y[:,i], y_pred_dt[:,i])
    auc_dt[i]= auc(fpr_dt[i], tpr_dt[i])
    fpr_nb[i], tpr_nb[i], thresholds_mlp = roc_curve(Y[:,i], y_pred_nb[:,i])
    auc_nb[i]= auc(fpr_nb[i], tpr_nb[i])
    fpr_xgb[i], tpr_xgb[i], thresholds_svm = roc_curve(Y[:,i], y_pred_xgb[:,i])
    auc_xgb[i]= auc(fpr_xgb[i], tpr_xgb[i])
    #colors = cycle(['r', 'b', 'tab:olive','c','k','g','y','tab:pink','tab:orange','tab:purple','m','tab:brown'])
    colors = cycle(['b', 'r', 'c'])
    colors1 = cycle(['k','g','y'])
    colors2 = cycle(['m','tab:pink' ,'tab:brown'])
    colors3 = cycle(['tab:olive','tab:orange','tab:purple'])



for i, color in zip(range(3), colors):
    plt.plot( fpr_bc[i], tpr_bc[i], color=color,
             label='CNN+BC class %i (area = %0.3f)'% (i,auc_bc[i]))
for i, color in zip(range(3), colors1):
    plt.plot( fpr_dt[i], tpr_dt[i], color=color,
             label='CNN+DT class %i (area = %0.3f)'% (i,auc_dt[i] ))
for i, color in zip(range(3), colors2):
    plt.plot( fpr_nb[i], tpr_nb[i], color=color,
             label='CNN+GNB class %i (area = %0.3f)'% (i,auc_nb[i] ))
for i, color in zip(range(3), colors3):
    plt.plot( fpr_xgb[i], tpr_xgb[i], color=color,
                label='CNN+XGBoost class %i (area = %0.3f)'% (i,auc_xgb[i] )) 
    
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('AUC-ROC Curve')
plt.legend(loc='best')

plt.show()


## RY and RX **Comparsion**

In [ ]:
plt.style.use('seaborn-white')

plt.figure(figsize=(7,7))
#plt.rcParams['axes.facecolor'] = 'k'
#plt.facecolor="w"
#plt.axes.color="k"
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_Q, tpr_Q,'m',label=' HQCNN +RY+500 shots (area = {:.3f})'.format(auc_Q))
plt.plot(fpr_cnn, tpr_cnn,'g', label=' HQCNN+RX+500 shots (area = {:.3f})'.format(auc_C))
plt.plot(fpr_Q2, tpr_Q2,'r',label=' HQCNN +RY+1000 shots (area = {:.3f})'.format(auc_Q2))
plt.plot(fpr_cnn2, tpr_cnn2,'b', label=' HQCNN+RX+1000 shots (area = {:.3f})'.format(auc_C2))
#plt.plot(fpr_mlp, tpr_mlp,'tab:olive', label=' MLP (area = {:.3f})'.format(auc_mlp))
#plt.plot(fpr_svm, tpr_svm,'c', label=' CNN+SVM (RBF) (area = {:.3f})'.format(auc_svm))
#plt.plot(fpr_svmP, tpr_svmP,'k', label=' CNN+SVM (Poly) (area = {:.3f})'.format(auc_svmP))
##plt.plot(fpr_knn, tpr_knn,'g', label=' CNN+KNN (area = {:.3f})'.format(auc_KNN))
#plt.plot(fpr_ABC, tpr_ABC,'y', label=' CNN+AdaBoost (area = {:.3f})'.format(auc_ABC))
#plt.plot(fpr_RF, tpr_RF,'tab:pink', label=' CNN+RF (area = {:.3f})'.format(auc_RF))
#plt.plot(fpr_mnb, tpr_mnb,'tab:orange', label=' CNN+BC (area = {:.3f})'.format(auc_mnb))
#plt.plot(fpr_DT, tpr_DT,'tab:purple', label=' CNN+DT (area = {:.3f})'.format(auc_DT))
#plt.plot(fpr_GNB, tpr_GNB,'m', label=' CNN+GNB (area = {:.3f})'.format(auc_GNB))
#plt.plot(fpr_XGB, tpr_XGB,'tab:brown', label=' CNN+XGBoost (area = {:.3f})'.format(auc_XGB))

plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('AUC-ROC Curve')
plt.legend(loc='best')

plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, label)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.style.use('seaborn-white')

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6 ,8))
epochs_=range(0,20)    
#ax1.set_ylim(top= 100) 
ax1.set_xlim(right=20) 
ax1.plot(epochs_,q_history.history["val_accuracy"],'b' , label=" Test Accuarcy + HQCNN+RY+500 shots ")
ax1.plot(epochs_,q_history2.history["val_accuracy"],'r',  label=" Test Accuarcy + HQCNN+RY+1000 shots ")
ax1.plot(epochs_,q_historyx.history["val_accuracy"], 'g', label=" Test Accuarcy + HQCNN+RX+500 shots ")
ax1.plot(epochs_,q_historyx2.history["val_accuracy"], 'c', label=" Test Accuarcy + HQCNN+RX+1000 shots")
ax1.set_ylabel("Accuracy")
#ax1.set_ylim(top=100)

ax1.set_xlabel("Epochs")
ax1.legend(loc="best")

ax2.plot(epochs_,q_history.history["val_loss"], 'b', label=" Test Loss + HQCNN+RY+500 shots")
ax2.plot(epochs_,q_history2.history["val_loss"], 'r', label=" Test Loss + HQCNN+RY+1000 shots")
ax2.plot(epochs_,q_historyx.history["val_loss"], 'g', label=" Test Loss + HQCNN+RX+500 shots")
ax2.plot(epochs_,q_historyx2.history["val_loss"],'c',  label=" Test Loss+ HQCNN+RX+1000 shots")
ax2.set_ylabel("Loss")
ax2.set_ylim(top=.5)
ax2.set_xlim(left=1,right=20) 

#ax2.set_xlim(right=20)
ax2.set_xlabel("Epochs")
ax2.legend()
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve
Q_probs = q_model.predict_proba(q_test_imagesZ)[:,1]
C_probs = VCmodel.predict_proba(xxx_ttest)[:,1]
svm_probs = svm.predict_proba(feat_test)[:,1]
svmP_probs = svmPoly.predict_proba(feat_test)[:,1]
knn_probs = neigh.predict_proba(feat_test)[:,1]
abc_probs = ABC.predict_proba(feat_test)[:,1]
rf_probs = RF.predict_proba(feat_test)[:,1]
dt_probs = DT.predict_proba(feat_test)[:,1]
gnb_probs = gnb.predict_proba(feat_test)[:,1]
xgb_probs = xgbc.predict_proba(feat_test)[:,1]


# keep probabilities for the positive outcome only
#Q_probs = Q_probs[:, 1]
#C_probs = C_probs[:, 1]

# calculate scores
C_auc = roc_auc_score(y_test, C_probs)
Q_auc = roc_auc_score(y_test, Q_probs)
# calculate precision and recall for each threshold
Q_precision, Q_recall, _ = precision_recall_curve(y_test,Q_probs)
C_precision, C_recall, _ = precision_recall_curve(y_test,C_probs)
svm_precision, svm_recall, _ = precision_recall_curve(y_test,svm_probs)
svmP_precision, svmP_recall, _ = precision_recall_curve(y_test,svmP_probs)
knn_precision, knn_recall, _ = precision_recall_curve(y_test,knn_probs)
abc_precision, abc_recall, _ = precision_recall_curve(y_test,abc_probs)
rf_precision, rf_recall, _ = precision_recall_curve(y_test,rf_probs)
dt_precision, dt_recall, _ = precision_recall_curve(y_test,dt_probs)
gnb_precision, gnb_recall, _ = precision_recall_curve(y_test,gnb_probs)
xgbprecision, xgbrecall, _ = precision_recall_curve(y_test,xgb_probs)

# calculate scores

# summarize scores

plt.figure(figsize=(8,8))
plt.title(' Precision-Recall Curve')
# plot the precision-recall curves
#no_skill = len(testy[testy==1]) / len(testy)
#pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
plt.plot(Q_recall, Q_precision, "m",  label='HQCNN')
plt.plot(C_recall, C_precision,'b', label='CNN')
plt.plot(svm_recall, svm_precision, "c",  label='CNN+SVM(RBF)')
plt.plot(svmP_recall, svmP_precision,'k', label='CNN+SVM(POLY)')
plt.plot(knn_recall, knn_precision, "g",  label='CNN+KNN')
plt.plot(abc_recall, abc_precision,'y', label='CNN+AdaBoost')
plt.plot(rf_recall, rf_precision, "tab:pink",  label='CNN+RF')
plt.plot(dt_recall, dt_precision,'tab:purple', label='CNN+DT')
plt.plot(gnb_recall, gnb_precision, "r",  label='CNN+GNB')
plt.plot(xgbrecall, xgbprecision,'tab:cyan', label='CNN+XGBoost')
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
# show the legend
plt.legend(loc='best')
# show the plot
plt.show()
print(Q_auc)

In [ ]:


from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

# predict probabilities
Q_probs = q_model.predict_proba(q_test_imagesZ)
C_probs = VCmodel.predict_proba(test_imgs)

# keep probabilities for the positive outcome only
Q_probs = Q_probs[:, 1]
C_probs = C_probs[:, 1]

# calculate scores
C_auc = roc_auc_score(y_test_PP, C_probs)
Q_auc = roc_auc_score(y_test_PP, Q_probs)
# summarize scores
print(' QCNN: ROC AUC=%.3f' % (Q_auc))
print(' CNN: ROC AUC=%.3f' % (C_auc))

# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(y_test_PP, C_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_test_PP, Q_probs)
# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='CNN')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='QCNN')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()